In [1]:
import os

In [2]:
%pwd

'e:\\Data_Science\\Text_summarization\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Data_Science\\Text_summarization'

In [5]:
## update config.yaml

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [7]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [9]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

[2025-01-21 20:01:18,988: INFO: config: PyTorch version 2.5.1 available.]


In [ ]:
import logging
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from transformers import DataCollatorForSeq2Seq
import torch
import os
from datasets import load_from_disk

class ModelTrainer:
    def __init__(self, config):
        self.config = config

    def freeze_layers(self, model, freeze_all_except_last=True):
        """ Freeze all layers except for the last one """
        for name, param in model.named_parameters():
            if freeze_all_except_last and 'encoder' in name:  # Freeze encoder layers
                param.requires_grad = False
            elif freeze_all_except_last and 'decoder' in name:  # Freeze decoder layers
                param.requires_grad = False
            else:
                param.requires_grad = True  # Leave the last layer trainable

    def train(self):
        logging.info("Starting training process...")
        os.environ["TRANSFORMERS_CACHE"] = "C:/Users/ADITYA/.cache"
        device = "cpu"  # Explicitly use CPU
        torch.set_num_threads(4)  # Limit to 4 threads (can increase if you have more cores)
        logging.info("Using device: %s", device)

        # Load tokenizer and model
        logging.info("Loading tokenizer and model from checkpoint: %s", self.config.model_ckpt)
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)

        # Freeze layers
        self.freeze_layers(model)

        # Data collator for padding and batch creation
        logging.info("Preparing data collator.")
        data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
        
        # Load dataset and select a subset
        logging.info("Loading dataset from: %s", self.config.data_path)
        dataset = load_from_disk(self.config.data_path)
        dataset["train"] = dataset["train"].select(range(6))  # Train on 6 samples
        dataset["validation"] = dataset["validation"].select(range(2))  # Validate on 2 samples
        logging.info("Dataset loaded. Train size: %d, Validation size: %d", len(dataset["train"]), len(dataset["validation"]))

        # Define training arguments with optimization
        logging.info("Setting up training arguments.")
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=0.1,
            per_device_train_batch_size=1,
            per_device_eval_batch_size=1,
            save_steps=1000,
            save_total_limit=1,
            evaluation_strategy='steps',
            logging_steps=500,
            gradient_accumulation_steps=8,
            fp16=True,  # Mixed precision
            remove_unused_columns=True,
            load_best_model_at_end=True,  # Save best model during training
            dataloader_num_workers=4,  # Parallelize data loading if possible
            run_name="optimized_training_run",  # Optional: Track the run in a specific way
        )
        logging.info("Training arguments configured.")

        # Enable gradient checkpointing if supported
        model.gradient_checkpointing_enable()

        # Initialize the Trainer
        logging.info("Initializing Trainer.")
        trainer = Trainer(
            model=model,
            args=trainer_args,
            tokenizer=tokenizer,
            data_collator=data_collator,
            train_dataset=dataset["train"],
            eval_dataset=dataset["validation"],
        )

        # Train the model
        logging.info("Starting training...")
        try:
            trainer.train()
            logging.info("Training completed successfully.")
        except Exception as e:
            logging.error("An error occurred during training: %s", str(e))
            raise

        # Save the trained model and tokenizer
        logging.info("Saving the trained model and tokenizer.")
        model.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))
        logging.info("Model and tokenizer saved to %s.", self.config.root_dir)

# Main script to execute training
try:
    logging.info("Initializing ConfigurationManager...")
    # Initialize the configuration manager
    config = ConfigurationManager()
    
    # Get the model trainer configuration
    model_trainer_config = config.get_model_trainer_config()
    logging.info("Configuration loaded successfully.")

    # Pass the configuration to the ModelTrainer
    model_trainer = ModelTrainer(config=model_trainer_config)
    
    # Train the model
    logging.info("Starting the model training process.")
    model_trainer.train()
except Exception as e:
    # Print the error message for debugging
    logging.critical("Critical error in training pipeline: %s", str(e))
    raise


[2025-01-21 20:01:21,156: INFO: 1061650405: Initializing ConfigurationManager...]
[2025-01-21 20:01:21,164: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-21 20:01:21,170: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-21 20:01:21,173: INFO: common: created directory at: artifacts]


[2025-01-21 20:01:21,173: INFO: common: created directory at: artifacts/model_trainer]
[2025-01-21 20:01:21,173: INFO: 1061650405: Configuration loaded successfully.]
[2025-01-21 20:01:21,181: INFO: 1061650405: Starting the model training process.]
[2025-01-21 20:01:21,184: INFO: 1061650405: Starting training process...]
[2025-01-21 20:01:21,457: INFO: 1061650405: Using device: cpu]
[2025-01-21 20:01:21,459: INFO: 1061650405: Loading tokenizer and model from checkpoint: google/pegasus-cnn_dailymail]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[2025-01-21 20:01:37,388: INFO: 1061650405: Preparing data collator.]
[2025-01-21 20:01:37,388: INFO: 1061650405: Loading dataset from: artifacts/data_transformation/samsum_dataset]
[2025-01-21 20:01:38,437: INFO: 1061650405: Dataset loaded. Train size: 6, Validation size: 2]
[2025-01-21 20:01:38,437: INFO: 1061650405: Setting up training arguments.]
[2025-01-21 20:01:38,462: INFO: 1061650405: Training arguments configured.]
[2025-01-21 20:01:38,467: INFO: 1061650405: Initializing Trainer.]
[2025-01-21 20:01:38,506: INFO: 1061650405: Starting training...]


c:\Users\ADITYA\anaconda3\envs\TextSummarization\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\ADITYA\AppData\Local\Temp\ipykernel_12184\1061650405.py:73: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss


c:\Users\ADITYA\anaconda3\envs\TextSummarization\Lib\site-packages\transformers\modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


[2025-01-21 20:07:13,500: INFO: 1061650405: Training completed successfully.]
[2025-01-21 20:07:13,974: INFO: 1061650405: Saving the trained model and tokenizer.]
[2025-01-21 20:10:35,810: INFO: 1061650405: Model and tokenizer saved to artifacts/model_trainer.]


In [11]:
'''
try:
    # Initialize the configuration manager
    config = ConfigurationManager()
    
    # Get the model trainer configuration
    model_trainer_config = config.get_model_trainer_config()
    
    # Pass the configuration to the ModelTrainer
    model_trainer = ModelTrainer(config=model_trainer_config)  # Changed variable to `model_trainer`
    
    # Set Hugging Face cache directory
    os.environ["TRANSFORMERS_CACHE"] = "C:/Users/ADITYA/.cache"  # Replace with your cache directory
    
    # Train the model
    model_trainer.train()
except Exception as e:
    # Print the error message for debugging
    print(f"An error occurred: {e}")
    raise e'''

'\ntry:\n    # Initialize the configuration manager\n    config = ConfigurationManager()\n    \n    # Get the model trainer configuration\n    model_trainer_config = config.get_model_trainer_config()\n    \n    # Pass the configuration to the ModelTrainer\n    model_trainer = ModelTrainer(config=model_trainer_config)  # Changed variable to `model_trainer`\n    \n    # Set Hugging Face cache directory\n    os.environ["TRANSFORMERS_CACHE"] = "C:/Users/ADITYA/.cache"  # Replace with your cache directory\n    \n    # Train the model\n    model_trainer.train()\nexcept Exception as e:\n    # Print the error message for debugging\n    print(f"An error occurred: {e}")\n    raise e'

In [12]:
'''
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e
'''

'\ntry:\n    config = ConfigurationManager()\n    model_trainer_config = config.get_model_trainer_config()\n    model_trainer_config = ModelTrainer(config=model_trainer_config)\n    model_trainer_config.train()\nexcept Exception as e:\n    raise e\n'

In [13]:
'''
import os
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

# Set custom cache directory
cache_dir = "E:/Data science/transformers_cache"
os.environ["TRANSFORMERS_CACHE"] = cache_dir

try:
    # Initialize ConfigurationManager and get model trainer config
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()

    # Load Pegasus model and tokenizer from the same cache directory
    model_name = "google/pegasus-cnn_dailymail"
    tokenizer = PegasusTokenizer.from_pretrained(model_name, cache_dir=cache_dir)
    model = PegasusForConditionalGeneration.from_pretrained(model_name, cache_dir=cache_dir)

    # Pass the configuration to the ModelTrainer and train the model
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()

except Exception as e:
    raise e
'''

'\nimport os\nfrom transformers import PegasusForConditionalGeneration, PegasusTokenizer\n\n# Set custom cache directory\ncache_dir = "E:/Data science/transformers_cache"\nos.environ["TRANSFORMERS_CACHE"] = cache_dir\n\ntry:\n    # Initialize ConfigurationManager and get model trainer config\n    config = ConfigurationManager()\n    model_trainer_config = config.get_model_trainer_config()\n\n    # Load Pegasus model and tokenizer from the same cache directory\n    model_name = "google/pegasus-cnn_dailymail"\n    tokenizer = PegasusTokenizer.from_pretrained(model_name, cache_dir=cache_dir)\n    model = PegasusForConditionalGeneration.from_pretrained(model_name, cache_dir=cache_dir)\n\n    # Pass the configuration to the ModelTrainer and train the model\n    model_trainer_config = ModelTrainer(config=model_trainer_config)\n    model_trainer_config.train()\n\nexcept Exception as e:\n    raise e\n'